# Анализ пользовательского взаимодействия с карточками статей для Яндекс.Дзен

Заказчик исследования: менеджеры контента Яндекс.Дзен

Задачи:

- проанализировать файл с событиями, созданный на основе пайплайна, разработанного дата-инженерами компании. Пайплайн работает с сырыми данными, агрегирует их и помещает в базу данных Zen

- подготовить ответы на вопросы менеджеров по анализу контента:
    - сколько взаимодействий пользователей с карточками происходит в системе с разбивкой по темам карточек?
    - как много карточек генерируют источники с разными темами?
    - как соотносятся темы карточек и темы источников?

- создать дашборд с результатами исследования на основе согласованного с менеджерами макета
- подготовить краткую презентацию 

Итак, установим необходимые библиотеки

In [1]:
pip install sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
from sqlalchemy import create_engine

Создаем подключение к базе данных

In [4]:
db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-zen-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                db_config['pwd'],
                                                db_config['host'],
                                                db_config['port'],
                                                db_config['db'])

engine = create_engine(connection_string)

Создаем запрос к базе данных и сохраняем его в csv-файл

In [5]:
query = '''
            SELECT * FROM dash_visits
        '''
df = pd.io.sql.read_sql(query, con = engine) 
df

,record_id,item_topic,source_topic,age_segment,dt,visits
0,1040597,Деньги,Авто,18-25,2019-09-24 18:32:00,3
1,1040598,Деньги,Авто,18-25,2019-09-24 18:35:00,1
2,1040599,Деньги,Авто,18-25,2019-09-24 18:54:00,4
3,1040600,Деньги,Авто,18-25,2019-09-24 18:55:00,17
4,1040601,Деньги,Авто,18-25,2019-09-24 18:56:00,27
...,...,...,...,...,...,...
30740,1071337,Юмор,Финансы,36-40,2019-09-24 18:57:00,2
30741,1071338,Юмор,Финансы,36-40,2019-09-24 19:00:00,1
30742,1071339,Юмор,Финансы,41-45,2019-09-24 18:54:00,1
30743,1071340,Юмор,Финансы,41-45,2019-09-24 18:56:00,1


In [8]:
df.to_csv('dash_visits.csv', index=False)
dash_visits = pd.read_csv('dash_visits.csv', sep=',')

In [9]:
dash_visits.head()

,record_id,item_topic,source_topic,age_segment,dt,visits
0,1040597,Деньги,Авто,18-25,2019-09-24 18:32:00,3
1,1040598,Деньги,Авто,18-25,2019-09-24 18:35:00,1
2,1040599,Деньги,Авто,18-25,2019-09-24 18:54:00,4
3,1040600,Деньги,Авто,18-25,2019-09-24 18:55:00,17
4,1040601,Деньги,Авто,18-25,2019-09-24 18:56:00,27


Посмотрим на структуру данных

In [10]:
dash_visits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30745 entries, 0 to 30744
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   record_id     30745 non-null  int64 
 1   item_topic    30745 non-null  object
 2   source_topic  30745 non-null  object
 3   age_segment   30745 non-null  object
 4   dt            30745 non-null  object
 5   visits        30745 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 1.4+ MB


Таблица состоит из 30745 строк, организованных в 6 столбцов. Пропуски отсутствуют, проблем с типами данных также нет.  

Структура данных:
- `record_id` - идентификатор записи  
- `item_topic` - тема карточки  
- `source_topic`- тема источника  
- `age_segment` - возрастная категория  
- `dt` - дата и время события  
- `visits` - количество событий

In [11]:
dash_visits['dt'].min()
dash_visits['dt'].max()

'2019-09-24 18:28:00'

'2019-09-24 19:00:00'

Записи лога охватывают период с 18:28 до 19:00 24 сентября 2019 года

## Выводы 

В нашем распоряжении были данные о взаимодействии пользователей с карточками статей на Яндекс.Дзен на дату 24.09.2021 за временной отрезок с 18:28 до 19:00. 

Пик пользовательской активности на сайте пришелся на промежуток с 18:56 до 18:59. В качестве метрики для оценки популярности тем карточек была выбрана медиана количества событий за исследуемый период. Рейтинг тем в порядке убывания интереса пользователей в период пиковой активности:

- отношения  
- наука    
- интересные факты  
- подборки  
- общество   
- Россия  

В совокупности на 8 самых популярных тем карточек из 25 приходится половина всех пользовательских событий.

Доли за период с 18:28 до 19:00 распределились следующим образом:  

- отношения (6,72%)
- наука (6,7%)  
- общество (6,46%)
- интересные факты (6,23%)          
- подборки (5,87%)   
- Россия (5,59%)  
- полезные советы (5,12%)  
- история (4,94%)

Самые популярные каналы: 

- семейные отношения (10,74%)
- Россия (9,62%)
- полезные советы (8,84%)
- путешествия (7,78%)
- знаменитости (7,72%)

Рейтинг популярности тем в соотношении карточка - источник:

1. Рассказы - путешествия  
2. Общество - Россия  
3. Наука - кино  
4. Россия - Россия  
5. Подборки - полезные советы  
6. Общество - семейные отношения  
7. Отношения - полезные советы  

Анализ количества событий в разбивке по темам карточек для разных возрастных категорий показал следующие результаты:  
Все возрастные категории более всего интересуются темой отношений. Для пользователей старше 25 лет также актуальны полезные советы и рубрика "Подборки", тогда как в круг интересов читателей до 25 лет и 45+ входят также темы "общество" и "наука".
У группы совершеннолетних пользователей до 25 лет распределение долей тем карточек более-менее стабильно на протяжении всего наблюдаемого периода. У остальных групп отмечаются всплески интереса к той или иной теме: 26-30 и 31-35 - интересные факты, психология, прутешествия, 36-40 - скандалы и здоровье, 41-45 - история и знаменитости, а 45+ - история и искусство.

Среди каналов наибольшей популярностью у всех возрастных групп пользуются темы "Семейные отношения" и "Россия". Пользователям старше 30 лет интересны также темы, посвященные жизни знаменитостей. 

Дашборд с результатами анализа находится здесь:  <code>[Дашборд](https://public.tableau.com/app/profile/oksana.golovina/viz/zen_dashboard_16288894207980/_?publish=yes)
</code>